<h1>Two-levels cross-validation for ANN model<h1>

Follow our project description two-level cross-validation, 
- Outer fold should be separated to train dataset and test dataset.
- Inner fold training datasets should be divided to training and validation sets.
- In inner fold, we should select the best hidden units to minimize the average validatation errors.
- As we have already selected the best hidden units in each group, then compare each other by calculating the Generalization in outer fold.


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt 
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader

Load the dataframe from the csv files we stored

In [5]:
train_x = pd.read_csv("/Users/luchengliang/02450_project2/2023-10-05_jennifer_data_preparation/independent_train.csv")
train_y = pd.read_csv("/Users/luchengliang/02450_project2/2023-10-05_jennifer_data_preparation/dependent_train.csv")
test_x = pd.read_csv("/Users/luchengliang/02450_project2/2023-10-05_jennifer_data_preparation/independent_test.csv")
test_y = pd.read_csv("/Users/luchengliang/02450_project2/2023-10-05_jennifer_data_preparation/dependent_test.csv")

To clarify if we used the GPUs (Mac will choose mps and Windows will choose cuda for GPUs) or CPU.

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [7]:
class ANN_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_hidden_layers):
        super(ANN_Model, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_hidden_layers = num_hidden_layers
        self.flatten = nn.Flatten()
        
        self.input_layer = nn.Linear(self.input_dim, self.hidden_dim)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(self.hidden_dim, self.hidden_dim) for _ in range(self.num_hidden_layers)
        ])
        self.output_layer = nn.Sequential(
            nn.Linear(self.hidden_dim, 1), 
            nn.Sigmoid()  
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.output_layer(x)
        return x
    